In [ ]:
# default_exp main

# Main

> API details.

In [ ]:
#export

from celverapi.core import *
import pandas as pd
import concurrent.futures 

In [ ]:
#export

# ---- Loading DataFrame

df = pd.read_parquet('C:/Users/kevinkeipert/Desktop/Data/example.parquet')

# ---- Project Variables 

id_column = "store_nbr"         #Name of the Column where the Product IDs are located.
                                #This will split the DataFrame into smaller DataFrames from every single unique value in the column.

date_column = "date"               #Name of the Colum where the Date is located. 

date_to_split = "2017-08-01"    #YYYY-MM-DD from where we want to split the DataFrames into Training and Test sets. 
                                #In this example, the Train set will be every day until 2017-08-01. And the Test set from 2017-08-01 (inclusive), until the end of the DataFrame.

value_column = "sales"             #Name of the Column where the values to messure are located. 
                                #In this example is the sales Column that contains the amount of money per day. 

forecast_horizon = 15           #Amount of steps to forecast 

In [ ]:
#export 

def unique_dataframe_values(id_column: str):
    
    """
    This Function returns a List of DataFrames from every unique Value in the ID Column.

    -   id_column (String) : Name of the Column from the DataFrame where the Unique IDs are located. This will split the DataFrame into smaller DataFrames from every single unique value in the column.
    
    """

    list_of_df = []

    array_uvalues = df[id_column].unique()

    for value in array_uvalues: 

        df_value = df[df[id_column] == value]
        list_of_df.append(df_value)

    return list_of_df 

list_of_df = unique_dataframe_values(id_column)

In [ ]:
#export

def split_into_train_test_dataframe(list_of_df: list, date_column: str, date_to_split: str):

    """
    The function splits the DataFrames into a Traing and Test sets.
    The DataFrames are saved in 2 Differents Lists, one for TRAIN DF and other for TEST DF.
    It returs two lists: list_of_trained_df, list_of_test_df

    -   list_of_df (List) : Lists of DataFrames with only one ID.
    -   date_column (String) : Name of the Colum where the Date is located.
    -   date_to_split (String) : YYYY-MM-DD from where we want to split the DataFrames into Training and Test sets. Example, the Train set will be every day until 2017-08-01. And the Test set from 2017-08-01 (inclusive), until the end of the DataFrame.
    
    """
    
    list_df_train = []
    list_df_val = []
    
    for df in list_of_df:
    
        df_train = df[df[date_column] < date_to_split]
        df_val = df[df[date_column] >= date_to_split]

        list_df_train.append(df_train)
        list_df_val.append(df_val)

    return list_df_train, list_df_val

list_df_train, list_df_val = split_into_train_test_dataframe(list_of_df, date_column, date_to_split)
    

In [ ]:
#export

def converting_dataframes_into_objects(list_df_train: list, list_df_val: list, date_column: str, id_column: str, value_column: str):

    """
    The function creates TimeSeries Object from the respective DataFrames
    The Objects are saved in 2 Differents Lists, one for TRAIN DF and other for TEST DF.
    It returs two lists: trained_timeseries, valed_timeseries

    -   list_df_train : List of trained DataFrames
    -   list_df_val (List) : List of tested DataFrames 
    -   date_column (String) : Name of the Colum where the Date is located.
    -   id_column (String) : Name of the Column from the DataFrame where the Unique IDs are located.
    -   value_column (String) : Name of the Column where the values to messure are located. 
    
    """
    
    trained_ts = []
    valed_ts = []

    for df in list_df_train:

        ts_train = TimeSeries(df, date_column, id_column, value_column)
        trained_ts.append(ts_train)

    for df in list_df_val:

        ts_val = TimeSeries(df, date_column, id_column, value_column)
        valed_ts.append(ts_val)


    return trained_ts, valed_ts

trained_ts, valed_ts= converting_dataframes_into_objects(list_df_train, list_df_val, date_column, id_column, value_column)

In [ ]:
#export

def creating_forecasting_task_dict(trained_ts: list, valed_ts: list, forecast_horizon: int):

    """
    The function creates a dictionary mapping the ID value with the respective ForecastingTask object. 
    It returs a dictionary : dict_of_forecasting_tasks 

    -   trained_ts (List) : List of trained Timeseries objects. 
    -   valed_ts (List) : List of tested/validated objects. 
    -   forecast_horizon (int) : Amount of steps to forecast 
    
    """

    dict_of_forecasting_tasks = {} 

    for i in range(len(trained_ts)):

        ft = ForecastingTask(train=trained_ts[i], out_of_sample=None, validation=valed_ts[i], forecast_horizon= forecast_horizon)
        dict_of_forecasting_tasks[trained_ts[i].get_id()] = ft

    return dict_of_forecasting_tasks


dict_of_forecasting_tasks = creating_forecasting_task_dict(trained_ts, valed_ts, forecast_horizon=15)

In [ ]:
#export 


def ets_func(ft: ForecastingTask):
    
    """
    Function for the ETS Model. 
    It returns a DataFrame with the following columns: Qty(Predicted), ID, Date, Model

    - ft : ForecastingTask Object 
    
    """

    ets = ETSModel(ft)
    ets.fit()
    prediction = ets.predict()
    id = ft.train.get_id()
    datum = ft.validation.get_datum()

    final_dataframe = prediction.to_frame(name='Qty')
    final_dataframe['ID'] = id 
    final_dataframe['Date'] = datum
    final_dataframe['Model'] = 'ETS-Model'
    
    return final_dataframe

def naive_func(ft: ForecastingTask):

    """
    Function for the Naive Model. 
    It returns a DataFrame with the following columns: Qty(Predicted), ID, Date, Model

    - ft : ForecastingTask Object 

    """

    naive = NaiveForecast(ft)
    naive.fit()
    prediction = naive.predict()
    id = ft.train.get_id()
    datum = ft.validation.get_datum()

    final_dataframe = prediction.to_frame(name='Qty')
    final_dataframe['ID'] = id 
    final_dataframe['Date'] = datum
    final_dataframe['Model'] = 'Naive Model'
    
    return final_dataframe

def trend_func(ft: ForecastingTask):

    """
    Function for the Trend Model. 
    It returns a DataFrame with the following columns: Qty(Predicted), ID, Date, Model

    - ft : ForecastingTask Object 

    """

    theta = TrendForecast(ft)
    theta.fit()
    prediction = theta.predict()
    id = ft.train.get_id()
    datum = ft.validation.get_datum()

    final_dataframe = prediction.to_frame(name='Qty')
    final_dataframe['ID'] = id 
    final_dataframe['Date'] = datum
    final_dataframe['Model'] = 'Trend Model'
    
    return final_dataframe


In [ ]:
#export 
def forecast(list_of_models: list, dict_of_forecasting_tasks: dict):

    """
    This Function allows the user to forecast multiple Timeseires and multiple Models simultaneously using a multiprocessing aproach. 
    It returns a DataFrame with all the models and Timeseries passed in the function.

    -   list_of_models(List) : A list with the Forecasting models to apply. For the moment only "ets", "naive" and trend".
    -   dict_of_forecasting_tasks(Dict) : A dictionary with the Forecasting tasks. Can be created with the function "creating_forecasting_task_dict()". 

    > Important: This function needs to run inside of the if __name__ == '__main__' condition.  

    """
    

    with concurrent.futures.ProcessPoolExecutor() as executor: 

        # List with all the Forecasted Objects 
        final_results_ets = []
        final_results_naive = []
        final_results_trend = []


        final_df = []

        # Run, for example various TimeSeries simultainly for Arima Model 

        if "ets" in list_of_models:

            ets_prediction = executor.map(ets_func, dict_of_forecasting_tasks.values())

            for result in ets_prediction:
                final_results_ets.append(result)
            
            result_ets = pd.concat(final_results_ets)
            final_df.append(result_ets)

        if "naive" in list_of_models:
            
            naive_prediction = executor.map(naive_func, dict_of_forecasting_tasks.values())

            for result in naive_prediction:
                final_results_naive.append(result) 

            result_naive = pd.concat(final_results_naive)
            final_df.append(result_naive)

        if "trend" in list_of_models:

            theta_prediction = executor.map(trend_func, dict_of_forecasting_tasks.values())

            for result in theta_prediction:
                final_results_trend.append(result)
        
            result_trend = pd.concat(final_results_trend)
            final_df.append(result_trend)

        result = pd.concat(final_df)


    return result
